In [1]:
import os
import numpy as np

from tensorflow.keras.models import model_from_json
import random
from IPython.display import clear_output
import time

In [2]:
def get_char_int_conversion():
    #basic characters in all books
    unique_chars = [' ', '!', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    #turn characters into integers and integers into characters - chars should be sorted to keep consistency
    char_to_int = {}
    int_to_char = {}
    for i, char in enumerate(sorted(set(unique_chars))):
        char_to_int[char] = i
        int_to_char[i] = char
        
    return char_to_int, int_to_char
#     print('number of unique characters: ', len(char_to_int)) #should be 48 every time
#     print('unique characters: ', list(char_to_int.keys()))

In [3]:
#returns a list of all the path weights and the architecture
def get_weights_and_arch():
    round_path = "models/"
    model_paths = []
    for root, dirs, files in os.walk(round_path):
        #file names in files path
        for file in files:
            if file.endswith('hdf5'):
                model_paths.append(root+"/"+file)
            elif file.endswith('model_8_architecture.json'):
                architecture_path = root+"/"+file
    #sort paths
    model_paths.sort()
#     print(architecture_path)
    
#     print(archi_paths[-1])
    # load json architecture
    json_file = open(architecture_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    #return model paths
    return model_paths, loaded_model_json

In [4]:
#creates new sentences
def create_passage(model, sentence):
#     sentence_length=50
#     #full passage string
#     sentence = create_sentence(sub_books_combined, sentence_length=sentence_length)
    sentence_length = len(sentence)
    
    print('original sentence: ', sentence.split("  ")[-1])
    print('\n')
    #predict the next 500 characters 
    gen_sentence = sentence
    print(gen_sentence.split("  ")[-1], end="", flush=True)
    for i in range(300):
#     period_cnt = 0
#     while period_cnt != 2:
        #turn single sentence into model format
        x_pred = np.zeros((1, sentence_length, len(char_to_int)))
        #fill tensor with correspoinding values
        for k, c in enumerate(sentence):
            x_ind = char_to_int[c]
            x_pred[0, k, x_ind] = 1

        #predict next character - returns predicted probabilities
        prob_c = model.predict(x_pred, verbose=0)[0]
        #turn to float64 - mulitnomial gives error otherwise
        prob_c = np.asarray(prob_c).astype('float64')
        #sample from the probability - try out difference softmax temperature values - 0.5 produced best results
        log_prob = np.log(prob_c) / 0.5
        exp_prob = np.exp(log_prob)
        pred_prob = exp_prob/np.sum(exp_prob)
        #sample from new probability distribution
        p = np.random.multinomial(1, pred_prob, 1) 
        #get the probability position
        prob_ind = np.argmax(p)
#         print(prob_ind)

        #turn prediction into a character
        next_c = int_to_char[prob_ind]
#         if next_c == '.':
#             period_cnt += 1
        #add character to generated sentence
        gen_sentence += next_c
        #get new sentence by sliding to index of sentence
        sentence = sentence[1:]+next_c
        print(next_c, end="", flush=True)

  
    return gen_sentence.split("  ")[-1]

In [5]:
def input_sentence():
    #take input from user
    while True:
        sentence = input("Type in some text: ")
        sent_len = len(sentence)
        #check if the user inputed anything
        if sent_len > 0:
            break
        else:
            print("You didn't enter anything in.")
            time.sleep(1.8)
            clear_output()
    
    #check if the length is less than the length of the sentences the model was trained on
    if sent_len < 50:
        while sent_len < 50:
            sentence = ' ' + sentence
            sent_len = len(sentence)
    #sentence has more characters than what was trained on
    else:
        #grab the last 50 characters
        sentence = sentence[-50:]
    
    #lowercase everything
    sentence = sentence.lower()
        
#     print('new_length: ', len(sentence))
#     print(sentence)
    
    return sentence

In [6]:
#get the char to int conversion and vice versa
char_to_int, int_to_char = get_char_int_conversion()

In [7]:
#get list of all models and the architecture
model_paths, loaded_model_json = get_weights_and_arch()

In [8]:
def main():
    #take input sentence from user
    sentence = input_sentence()
    clear_output()
    
    #load architecture
    loaded_model = model_from_json(loaded_model_json)
    #pick random weights from list
    path = random.choice(model_paths)
    # load weights into new model
    loaded_model.load_weights(path)
    print("\n Loaded model from disk: ", path)
    
    create_passage(loaded_model, sentence)

In [9]:
main()


 Loaded model from disk:  models/round8/weights-improvement-47-1.3478-bigger.hdf5
original sentence:  jeus


jeus, i serve the most some contempt of the consideration of the convent and fortune that the feet to so reason to be in the more when a reason, and the doubt of a man will see any one who would say at the point of the age of the rest that we have been a thing to take the course of the charge of his com